In [478]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import scipy.special

n_ops = 7
n_nodes = 3

alphas = []
alpha_pw_normal = []

norm_alphas = []

for i in range(n_nodes):
    a = nn.Parameter(1e-3 * torch.randn(i + 2, n_ops))
    num_comb = int(scipy.special.binom(i + 2, 2))
    alpha_pw_normal.append(nn.Parameter(1e-3 * torch.randn(num_comb)))
    
    alphas.append(a)
    norm_alphas.append(F.softmax(a, dim=-1))
    
    
alphas[0][0], norm_alphas[0][0]

(tensor([ 1.9719e-03, -2.2639e-04, -5.7323e-05,  3.8733e-04, -1.0382e-03,
          4.6009e-04, -1.5278e-04], grad_fn=<SelectBackward>),
 tensor([0.1431, 0.1428, 0.1428, 0.1429, 0.1427, 0.1429, 0.1428],
        grad_fn=<SelectBackward>))

In [482]:
int(scipy.special.binom(1 + 2, 2))

3

In [480]:
alphas

[Parameter containing:
 tensor([[ 1.9719e-03, -2.2639e-04, -5.7323e-05,  3.8733e-04, -1.0382e-03,
           4.6009e-04, -1.5278e-04],
         [ 7.9759e-04, -1.1061e-03,  8.7072e-04,  8.7971e-05,  7.9254e-04,
           2.2627e-04,  1.5051e-04]], requires_grad=True),
 Parameter containing:
 tensor([[ 1.4632e-03, -5.8189e-04, -1.0868e-03, -7.3043e-04,  8.1603e-04,
          -1.1836e-03, -1.0569e-03],
         [-1.0162e-03,  9.6582e-04,  1.7232e-03, -1.1815e-03,  1.0269e-03,
           3.7533e-04,  1.1480e-03],
         [-1.0145e-03, -7.9381e-04, -3.0377e-04,  9.3046e-04, -6.5374e-05,
           1.2392e-04,  1.6889e-03]], requires_grad=True),
 Parameter containing:
 tensor([[ 5.3051e-05, -4.3119e-04, -5.6323e-06, -1.6468e-04,  7.2717e-04,
           1.2750e-03,  6.7625e-04],
         [ 3.9846e-04, -3.0487e-04, -8.0514e-04,  1.9182e-03,  7.5514e-04,
           1.6797e-04,  1.8888e-04],
         [ 1.1912e-03,  1.1795e-03, -5.1644e-04, -9.3776e-04, -1.7412e-03,
          -6.6687e-04,  1.30

In [479]:
alpha_pw_normal

[Parameter containing:
 tensor([0.0004], requires_grad=True),
 Parameter containing:
 tensor([2.0163e-03, 1.5127e-03, 2.0280e-05], requires_grad=True),
 Parameter containing:
 tensor([-0.0005,  0.0003,  0.0018,  0.0009,  0.0004,  0.0004],
        requires_grad=True)]

In [53]:
C = calc_C_term(alphas[0][0])

# C_term
torch.sum(inverse_softmax(norm_alphas[0][0], C)), torch.sum(alphas[0][0])

(tensor(0.0013, grad_fn=<SumBackward0>),
 tensor(0.0013, grad_fn=<SumBackward0>))

In [54]:
inverse_softmax(norm_alphas[0][0], C), F.softmax(inverse_softmax(norm_alphas[0][0], C), dim=-1)

(tensor([ 0.0008,  0.0017, -0.0001,  0.0014, -0.0016, -0.0014,  0.0006],
        grad_fn=<AddBackward0>),
 tensor([0.1429, 0.1431, 0.1428, 0.1430, 0.1426, 0.1426, 0.1429],
        grad_fn=<SoftmaxBackward>))

In [28]:
## Now, increasing the alphas

In [29]:
increase_p = 0.3
norm_alphas[0][0][0] += increase_p

for i in range(1, 7):
    norm_alphas[0][0][i] -= increase_p/(n_ops-1)

norm_alphas[0][0]

tensor([0.4427, 0.0928, 0.0926, 0.0928, 0.0931, 0.0929, 0.0932],
       grad_fn=<SelectBackward>)

In [32]:
inverse_softmax(norm_alphas[0][0], C)

tensor([ 1.1305, -0.4320, -0.4345, -0.4315, -0.4288, -0.4309, -0.4276],
       grad_fn=<AddBackward0>)

In [455]:
def inverse_softmax(x, C):
    return torch.log(x) + C

def calc_C_term(a):
    e_xi = torch.exp(a)
    sum_i = torch.sum(e_xi)
    c_i = torch.log(sum_i)
    return c_i

# TODO: with torch no grad
# norm_alphas, alphas, n_ops
prob = 0.3


# Limits increase, 
def increase_op(row_idx, edge_idx, op_idx):
    C = calc_C_term(alphas[0][0])
    
    if norm_alphas[row_idx][edge_idx][op_idx] + prob < 1.0:
        with torch.no_grad():
            norm_alphas[row_idx][edge_idx][op_idx] += prob

        # Align the other probabilities by lowering them
        indices = np.delete(np.arange(n_ops), op_idx)
        with torch.no_grad():
            norm_alphas[0][0][indices] -= prob/(n_ops-1)
    
    with torch.no_grad():
        alphas[0][0] = inverse_softmax(norm_alphas[0][0], C)
    print(alphas[0][0])
    
    norm_alphas[0][0] = F.softmax(alphas[0][0], dim=-1)
    print(norm_alphas[0][0])
    # return alpha


In [92]:
increase_op(0, 0, 0)

tensor([ 1.1311, -0.4315, -0.4314, -0.4287, -0.4320, -0.4305, -0.4316],
       grad_fn=<SelectBackward>)
tensor([0.4428, 0.0928, 0.0928, 0.0931, 0.0928, 0.0929, 0.0928],
       grad_fn=<SelectBackward>)


In [95]:
increase_op(0, 0, 0)

tensor([ 1.6484, -1.2053, -1.2051, -1.1992, -1.2063, -1.2030, -1.2055],
       grad_fn=<SelectBackward>)
tensor([0.7428, 0.0428, 0.0428, 0.0431, 0.0428, 0.0429, 0.0428],
       grad_fn=<SelectBackward>)


In [45]:
np.delete(np.arange(n_ops), 2)

array([0, 1, 3, 4, 5, 6])

In [49]:
norm_alphas[0][0][np.delete(np.arange(n_ops), 2)] += 1

In [48]:
norm_alphas[0][0]

tensor([ 1.0429, -0.0069, -0.0070, -0.0074, -0.0070, -0.0073, -0.0072],
       grad_fn=<SelectBackward>)

In [50]:
norm_alphas[0][0]

tensor([ 2.0429,  0.9931, -0.0070,  0.9926,  0.9930,  0.9927,  0.9928],
       grad_fn=<SelectBackward>)

In [ ]:
# TODO: Test edge cases#
# TODO: Same starting temperature?

In [459]:

n_ops = 7
n_nodes = 3

alphas = []
norm_alphas = []


for i in range(n_nodes):
    a = nn.Parameter(
        1e-3 * torch.randn(i + 2, n_ops))
    alphas.append(a)
    norm_alphas.append(F.softmax(a, dim=-1))
    
    
alphas[0][0], norm_alphas[0][0]


def inverse_softmax(x, C):
    return torch.log(x) + C

def calc_C_term(a):
#     e_xi = torch.exp(a)
#     sum_i = torch.sum(e_xi)
#     c_i = torch.log(sum_i)
#     return c_i
    
    return torch.log(torch.tensor([10.]))[0]

# TODO: with torch no grad
# norm_alphas, alphas, n_ops


# Fully, allows increase 
def increase_op(row_idx, edge_idx, op_idx, prob=0.3, n_ops=7):
    C = calc_C_term(alphas[row_idx][edge_idx])
    
    # Set short-hands
#     curr_op = norm_alphas[row_idx][edge_idx][op_idx]
#     curr_edge = norm_alphas[row_idx][edge_idx]
    curr_op = torch.tensor([0.32, 0.05, 0.01, 0.03, 0.23, 0.36, 0.00])[op_idx]
    curr_edge = torch.tensor([0.32, 0.05, 0.01, 0.03, 0.23, 0.36, 0.00])
    
    
    # Allow for increasing to 0.99
    if curr_op + prob > 1.0:
        surplus = curr_op + prob - 0.99
        prob -= surplus
    
    if curr_op + prob < 1.0:
        # Increase chosen op
        with torch.no_grad():
            curr_op += prob

        # Align the other probabilities by lowering them
#         indices = np.delete(np.arange(n_ops), op_idx)
        with torch.no_grad():
            curr_edge += 0.01
    
    with torch.no_grad():
        alphas[row_idx][edge_idx] = inverse_softmax(curr_edge, C)
        
        
    # Display for testing
    print("orig", alphas[row_idx][edge_idx])
    norm_alphas[row_idx][edge_idx] = F.softmax(alphas[row_idx][edge_idx], dim=-1)
    print("norm", norm_alphas[row_idx][edge_idx], torch.sum(norm_alphas[row_idx][edge_idx]))
    
    
    
def decrease_op(row_idx, edge_idx, op_idx, prob=0.3, n_ops=7):
    C = calc_C_term(alphas[row_idx][edge_idx])
    
    # Set short-hands
    curr_op = norm_alphas[row_idx][edge_idx][op_idx]
    curr_edge = norm_alphas[row_idx][edge_idx]
    
    # Allow for increasing to 0.99
    if curr_op - prob < 0.0:
        surplus = prob - curr_op + 0.01
        print(surplus)
        prob -= surplus
        print(prob)
    
    if curr_op - prob > 0.0:
        # Increase chosen op
        with torch.no_grad():
            curr_op -= prob

        # Align the other probabilities by lowering them
        indices = np.arange(n_ops), op_idx
        with torch.no_grad():
            norm_alphas[row_idx][edge_idx] += 0.01
    
    with torch.no_grad():
        alphas[row_idx][edge_idx] = inverse_softmax(curr_edge, C)
        
        
    # Display for testing
    print("orig", alphas[row_idx][edge_idx])
    norm_alphas[row_idx][edge_idx] = F.softmax(alphas[row_idx][edge_idx], dim=-1)
    print("norm", norm_alphas[row_idx][edge_idx], torch.sum(norm_alphas[row_idx][edge_idx]))
    
increase_op(0, 0, 0)

orig tensor([ 1.1632, -0.5108, -1.6094, -0.9163,  0.8755,  1.3083, -2.3026],
       grad_fn=<SelectBackward>)
norm tensor([0.3019, 0.0566, 0.0189, 0.0377, 0.2264, 0.3491, 0.0094],
       grad_fn=<SelectBackward>) tensor(1.0000, grad_fn=<SumBackward0>)


tensor(2.3026)

In [401]:
test_a = torch.tensor([0.32, 0.05, 0.01, 0.03, 0.23, 0.36, 0.01])
# TODO: Prevent value from being 0
test_a

tensor([0.3200, 0.0500, 0.0100, 0.0300, 0.2300, 0.3600, 0.0000])

In [414]:

increase_op(0, 0, 5)

orig tensor([ 0.9531, -0.7517, -1.8503, -1.1571,  0.6346,  1.0401, -2.5434],
       grad_fn=<SelectBackward>)
norm tensor([0.3113, 0.0566, 0.0189, 0.0377, 0.2264, 0.3396, 0.0094],
       grad_fn=<SelectBackward>) tensor(1.0000, grad_fn=<SumBackward0>)


In [390]:
increase_op(0, 0, 2)

orig tensor([ 1.1318e+00,  1.3165e+00,  1.5151e+00,  2.2275e-03,  1.4901e-04,
        -9.8681e-04, -4.2415e-04], grad_fn=<SelectBackward>)
norm tensor([0.2016, 0.2425, 0.2958, 0.0652, 0.0650, 0.0649, 0.0650],
       grad_fn=<SelectBackward>) tensor(1.0000, grad_fn=<SumBackward0>)


In [400]:
increase_op(0, 0, 5)

orig tensor([2.0433e+00, 3.5510e+00, 1.5151e+00, 2.2273e-03, 1.4877e-04, 4.4984e+00,
        3.1836e+00], grad_fn=<SelectBackward>)
norm tensor([0.0473, 0.2136, 0.0279, 0.0061, 0.0061, 0.5510, 0.1479],
       grad_fn=<SelectBackward>) tensor(1.0000, grad_fn=<SumBackward0>)


In [465]:
# TODO: Test this
# for i in range(100000):
#     decrease_op(0, 0, 0)

In [476]:
norm_alphas

[tensor([[0.0188, 0.1635, 0.1635, 0.1635, 0.1635, 0.1635, 0.1635],
         [0.1427, 0.1428, 0.1429, 0.1430, 0.1428, 0.1429, 0.1429]],
        grad_fn=<CopySlices>),
 tensor([[0.1429, 0.1427, 0.1428, 0.1430, 0.1429, 0.1428, 0.1428],
         [0.1428, 0.1431, 0.1426, 0.1426, 0.1430, 0.1429, 0.1430],
         [0.1428, 0.1427, 0.1428, 0.1428, 0.1429, 0.1428, 0.1430]],
        grad_fn=<SoftmaxBackward>),
 tensor([[0.1430, 0.1429, 0.1429, 0.1429, 0.1428, 0.1426, 0.1430],
         [0.1431, 0.1428, 0.1426, 0.1430, 0.1428, 0.1427, 0.1429],
         [0.1428, 0.1428, 0.1430, 0.1428, 0.1428, 0.1430, 0.1429],
         [0.1428, 0.1430, 0.1429, 0.1429, 0.1428, 0.1429, 0.1426]],
        grad_fn=<SoftmaxBackward>)]

In [472]:
import copy
a = [copy.deepcopy(alpha.detach()) for alpha in norm_alphas]
a

[tensor([[0.0188, 0.1635, 0.1635, 0.1635, 0.1635, 0.1635, 0.1635],
         [0.1427, 0.1428, 0.1429, 0.1430, 0.1428, 0.1429, 0.1429]]),
 tensor([[0.1429, 0.1427, 0.1428, 0.1430, 0.1429, 0.1428, 0.1428],
         [0.1428, 0.1431, 0.1426, 0.1426, 0.1430, 0.1429, 0.1430],
         [0.1428, 0.1427, 0.1428, 0.1428, 0.1429, 0.1428, 0.1430]]),
 tensor([[0.1430, 0.1429, 0.1429, 0.1429, 0.1428, 0.1426, 0.1430],
         [0.1431, 0.1428, 0.1426, 0.1430, 0.1428, 0.1427, 0.1429],
         [0.1428, 0.1428, 0.1430, 0.1428, 0.1428, 0.1430, 0.1429],
         [0.1428, 0.1430, 0.1429, 0.1429, 0.1428, 0.1429, 0.1426]])]

In [473]:
b = a

b

[tensor([[0.0188, 0.1635, 0.1635, 0.1635, 0.1635, 0.1635, 0.1635],
         [0.1427, 0.1428, 0.1429, 0.1430, 0.1428, 0.1429, 0.1429]]),
 tensor([[0.1429, 0.1427, 0.1428, 0.1430, 0.1429, 0.1428, 0.1428],
         [0.1428, 0.1431, 0.1426, 0.1426, 0.1430, 0.1429, 0.1430],
         [0.1428, 0.1427, 0.1428, 0.1428, 0.1429, 0.1428, 0.1430]]),
 tensor([[0.1430, 0.1429, 0.1429, 0.1429, 0.1428, 0.1426, 0.1430],
         [0.1431, 0.1428, 0.1426, 0.1430, 0.1428, 0.1427, 0.1429],
         [0.1428, 0.1428, 0.1430, 0.1428, 0.1428, 0.1430, 0.1429],
         [0.1428, 0.1430, 0.1429, 0.1429, 0.1428, 0.1429, 0.1426]])]

In [474]:
b[0][0][0] = 1

In [475]:
a

[tensor([[1.0000, 0.1635, 0.1635, 0.1635, 0.1635, 0.1635, 0.1635],
         [0.1427, 0.1428, 0.1429, 0.1430, 0.1428, 0.1429, 0.1429]]),
 tensor([[0.1429, 0.1427, 0.1428, 0.1430, 0.1429, 0.1428, 0.1428],
         [0.1428, 0.1431, 0.1426, 0.1426, 0.1430, 0.1429, 0.1430],
         [0.1428, 0.1427, 0.1428, 0.1428, 0.1429, 0.1428, 0.1430]]),
 tensor([[0.1430, 0.1429, 0.1429, 0.1429, 0.1428, 0.1426, 0.1430],
         [0.1431, 0.1428, 0.1426, 0.1430, 0.1428, 0.1427, 0.1429],
         [0.1428, 0.1428, 0.1430, 0.1428, 0.1428, 0.1430, 0.1429],
         [0.1428, 0.1430, 0.1429, 0.1429, 0.1428, 0.1429, 0.1426]])]